In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from model import *

num_samples = 10
unlabel_prob = 0.95 # probability of labels to be unlabeled
percentile = 0     # when a graph is not given, construct a graph 
                    # dropping nodes under this percentile 

sigma = 1.5         # sigma hyperparameter for RBF kernel

hard = False


%matplotlib inline

In [2]:
true_labels, features, graph = Data.Data.load_cora(multiclass=True)
labels, is_labeled, labeled_indices, unlabeled_indices = utils.random_unlabel(true_labels,unlabel_prob=unlabel_prob,multiclass=True)
solution = true_labels[unlabeled_indices]
weights, graph   = utils.rbf_kernel(features,s=sigma,G=graph)
num_nodes = len(labels)
num_classes = true_labels.shape[1]

graph given


In [3]:
# baseline_prediction = np.rint(labels[unlabeled_indices])
# print("[Baseline]　Accuracy:", utils.accuracy(solution, baseline_prediction), 
#       "RMSE:", utils.rmse(solution, baseline_prediction))

In [3]:
lp = LP.LP()
lp_closed_prediction = lp.closed(labels,
                                 weights,
                                 labeled_indices,
                                 unlabeled_indices)
lp_closed_onehot = utils.prob_to_one_hot(lp_closed_prediction)
lp_iter_prediction = lp.iter(labels,
                             weights,
                             is_labeled,
                             num_iter=100)
lp_iter_onehot = utils.prob_to_one_hot(lp_iter_prediction)

In [4]:
print("[Closed]　Accuracy:", utils.accuracy(solution, lp_closed_onehot,multiclass=True), 
      "MSE:", utils.mse(solution, lp_closed_prediction),
      "Log:", utils.log_loss(solution, lp_closed_prediction,multiclass=True))
print("[Iter]　Accuracy:", utils.accuracy(solution, lp_iter_onehot[unlabeled_indices],multiclass=True), 
      "MSE:", utils.mse(solution, lp_iter_prediction[unlabeled_indices]),
      "Log:", utils.log_loss(solution, lp_iter_prediction[unlabeled_indices],multiclass=True))

[Closed]　Accuracy: 0.775 MSE: 0.0620488856757 Log: 0.960635245642
[Iter]　Accuracy: 0.774576271186 MSE: 0.063334587098 Log: 0.983047508429


In [5]:
data, validation_data = Data.Data.prepare(labels,labeled_indices,true_labels,2,num_classes,num_samples,num_nodes)

In [6]:
dlp = DeepLPT.DeepLPT(num_nodes,num_classes,weights,num_iter=100,lr=0.1)


In [9]:
dlp_rbf = DeepLP_RBFT.DeepLP_RBFT(num_nodes,num_classes,features, graph, sigma, num_iter=100, lr=0.01, loss_type='log',graph_sparse=True) # usually 1

In [10]:
dlp_rbf_prediction,_ = dlp_rbf.labelprop(validation_data)

{'true_loss': 0.97816938, 'true_accuracy': 0.77457625}


In [ ]:
dlp_rbf.train(data,validation_data,200)

In [6]:

def labelprop(data):
    _open_sess()
    l_o_loss      = (_calc_loss(y,yhat,masked)
                          + regularize * _regularize_loss())
    # evaluate performance
    loss          = _calc_loss(y,yhat,1-labeled)
    # labeled_loss  = _calc_loss(y,yhat,labeled)
    accuracy      = _calc_accuracy(y,yhat,1-labeled)
    true_loss     = _calc_loss(y,yhat,1-true_labeled)
    true_accuracy = _calc_accuracy(y,yhat,1-true_labeled)

    pred,l_o_loss_,loss_,accuracy_,true_loss_,true_accuracy_ = _eval([yhat,l_o_loss,loss,accuracy,true_loss,true_accuracy],data)
    metrics = {
        'loss': loss_,
        'labeled_loss': l_o_loss_,
        'accuracy': accuracy_,
        'true_loss': true_loss_,
        'true_accuracy': true_accuracy_
    }
    print(metrics)

    return pred, metrics

def plot():
    _plot_loss()
    _plot_accuracy()
    _plot_params()

def train(data,validation_data,epochs=1000):
    _open_sess()

    n = len(data['X'])
    losses = []
    labeled_losses = []
    accuracies = []
    true_losses = []
    true_accuracies = []
    _save(-1,data,validation_data,n)
    for epoch in range(epochs):
        # Train with each example
        _eval(update,data)
        print("--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()
        _save(epoch,data,validation_data,n)
        if epoch > 2:
            latest = labeled_losses[-1]
            prev   = labeled_losses[-2]
            if latest - prev < 0.001:
                stopping_count += 1
            else:
                stopping_count = 0
            if stopping_count > 4:
                print("early stopping after",epoch,"epochs")
                break

def _backwardprop(y,
                        yhat,
                        labeled,
                        true_labeled,
                        masked,
                        regularize,
                        lr):
    '''
    Backprop on unlabeled + masked labeled nodes.
    Calculate loss and accuracy for both train and validation dataset.
    '''
    # backward propagation
    l_o_loss      = _calc_loss(y,yhat,masked)
    objective     = l_o_loss + regularize * _regularize_loss()
    update        = tf.train.AdamOptimizer(lr).minimize(objective)

    # evaluate performance
    loss          = _calc_loss(y,yhat,1-labeled)
    # labeled_loss  = _calc_loss(y,yhat,labeled)
    accuracy      = _calc_accuracy(y,yhat,1-labeled)
    true_loss     = _calc_loss(y,yhat,1-true_labeled)
    true_accuracy = _calc_accuracy(y,yhat,1-true_labeled)

    metrics = {
        'loss': loss,
        'labeled_loss': l_o_loss,
        'accuracy': accuracy,
        'true_loss': true_loss,
        'true_accuracy': true_accuracy
    }

    return update, metrics

def _calc_accuracy(y,yhat,mask):

    yhat = tf.argmax(yhat,axis=0)
    y = tf.argmax(y,axis=0)
    mask = tf.reduce_max(mask,axis=0)
    acc_mat = mask * tf.cast(tf.equal(yhat,y),tf.float32)
    return tf.reduce_sum(acc_mat) / tf.count_nonzero(mask,dtype=tf.float32)

def _calc_loss(y,yhat,mask, eps=0.00001):
    
    if loss_type == 'mse':
        loss_mat = mask * ((y - yhat) ** 2)
        loss = tf.reduce_sum(loss_mat) / tf.count_nonzero(mask,dtype=tf.float32)
    if loss_type == 'log':
        yhat = tf.minimum(tf.maximum(yhat, eps), 1-eps)
        loss_mat = mask * y * tf.log(yhat) * (-1)
        loss = tf.reduce_sum(loss_mat) / tf.count_nonzero(mask*y,dtype=tf.float32)

    return loss

def _eval(vals,data):
    return sess.run(vals, feed_dict={X:data['X'],
                                          y:data['y'],
                                          labeled:data['labeled'],
                                          true_labeled:data['true_labeled'],
                                          masked:data['masked']})

def _forwardprop(X,
                       weights,
                       labeled,
                       num_iter):
    'Forward prop which mimicks LP.'

    Tnorm = _tnorm(weights)
    Tnorm_T = tf.transpose(Tnorm)

    def layer(i,h,X,Tnorm_T):
        # propagate labels
        h = tf.tensordot(h,
                         Tnorm_T,axes=1)
        h.set_shape(shape)
        # don't update labeled nodes
        h = h * (1-labeled) + X * labeled

        return [i+1,h,X,Tnorm_T]

    def condition(i,X,trueX,Tnorm):
        return num_iter > i

    _,yhat,_,_ = tf.while_loop(condition, layer, loop_vars=[0,X,X,Tnorm_T])
    return yhat

def _get_val(val):
    return sess.run(val)

def _init_weights(weights_np):
    'Weight initialization.'
    weights = tf.convert_to_tensor(weights_np, np.float32)
    return tf.Variable(weights)

def _open_sess():
    sess = tf.Session()
    init      = tf.global_variables_initializer()
    sess.run(init)

def _plot_accuracy():
    plt.plot(accuracies,label="DeepLP, train")
    plt.plot([true_accuracies[0]] * len(accuracies),label="LP")
    plt.plot(true_accuracies,label="DeepLP, validation")
    plt.title("accuracy")
    plt.legend()
    plt.show()

def _plot_loss():
    plt.plot(labeled_losses,label="labeled loss")
    plt.plot(losses,label="unlabeled loss")
    plt.plot(true_losses,label='validation unlabeled loss')
    plt.title("loss")
    plt.legend()
    plt.show()

def _plot_params():
    pass

def _regularize_loss():
    return 0
    # tf.nn.l2_loss(theta-1)

def _save_params(epoch,data,n):
    pass

def _save(epoch,data,validation_data,n):

    loss,labeled_loss,accuracy = _eval([metrics['loss'],metrics['labeled_loss'],metrics['accuracy']],data)
    true_loss,true_accuracy    = _eval([metrics['true_loss'],metrics['true_accuracy']],validation_data)
    losses.append(loss)
    labeled_losses.append(labeled_loss)
    accuracies.append(accuracy)
    true_losses.append(true_loss)
    true_accuracies.append(true_accuracy)

    if epoch % 1 == 0 or epoch == -1:
        print("epoch:",epoch,
              "labeled loss:",labeled_loss,
              "unlabeled loss:",loss,
              "accuracy:",accuracy,
              "true unlabeled loss:",true_loss,
              "true accuracy:",true_accuracy)
        start_time = time.time()
    _save_params(epoch,data,n)

def _tnorm(weights):
    Tnorm = weights / tf.reduce_sum(weights, axis = 1, keep_dims=True)
    return Tnorm


In [7]:
weights      = _init_weights(weights)

In [13]:
    # initialize placeholders
    shape             = [num_classes, None, num_nodes]
    X            = tf.placeholder("float", shape=shape)
    y            = tf.placeholder("float", shape=shape)
    labeled      = tf.placeholder("float", shape=shape)
    true_labeled = tf.placeholder("float", shape=shape)
    masked       = tf.placeholder("float", shape=shape)
    shape        = shape

    yhat         = _forwardprop(X,
                                          weights,
                                          labeled,
                                          100)
    loss_type = 'mse'

In [14]:
    l_o_loss      = (_calc_loss(y,yhat,masked)
                          + 0 * _regularize_loss())
    # evaluate performance
    loss          = _calc_loss(y,yhat,1-labeled)
    # labeled_loss  = _calc_loss(y,yhat,labeled)
    accuracy      = _calc_accuracy(y,yhat,1-labeled)
    true_loss     = _calc_loss(y,yhat,1-true_labeled)
    true_accuracy = _calc_accuracy(y,yhat,1-true_labeled)

In [15]:
data, validation_data = Data.Data.prepare(labels,labeled_indices,true_labels,2,num_classes,num_samples,num_nodes)

In [16]:
    yhat_max = tf.argmax(yhat,axis=0)
    y_max = tf.argmax(y,axis=0)
#     mask = tf.reduce_max(mask,axis=0)
#     acc_mat = mask * tf.cast(tf.equal(yhat,y),tf.float32)
#     return tf.reduce_sum(acc_mat) / tf.count_nonzero(mask,dtype=tf.float32)

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    loss_ = sess.run(true_loss, feed_dict={X:validation_data['X'],
                                              y:validation_data['y'],
                                              labeled:validation_data['labeled'],
                                              true_labeled:validation_data['true_labeled'],
                                              masked:validation_data['masked']})


In [18]:
loss_

0.060952485

In [26]:
np.count_nonzero(solution)

2360

In [30]:
np.count_nonzero(validation_data['masked'])

140

In [34]:
np.sum((1-validation_data['true_labeled']) * validation_data['y'])

2360.0

In [24]:
y_max_ == yhat_max

False

In [27]:
np.mean(yhat_max_[0][unlabeled_indices]==y_max_[0][unlabeled_indices])

0.78262711864406775

In [31]:
np.sum(validation_data['y'] > 1)

0

In [28]:
validation_data['true_labeled'][:,0,:].T[unlabeled_indices]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
utils.accuracy(solution, utils.prob_to_one_hot(yhat_[:,0,:].T[unlabeled_indices]),multiclass=True), 

(0.76864406779661021,)

In [55]:
y_max

array([[0, 0, 0, ..., 0, 0, 2]])

In [54]:
np.sum(validation_data['y'][:,0,:],axis=0)

array([ 2.,  0.,  0., ...,  1.,  0.,  2.])

In [65]:
validation_data['y'][:,0,:].T[unlabeled_indices]

array([[ 1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  1.,  0.,  0.]])

In [45]:
np.argmax(solution,axis=1)

array([0, 1, 2, ..., 5, 6, 0])

In [84]:
solution

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [70]:
true_labels

(2485, 7)

In [85]:
np.tile(true_labels.T,1).reshape(num_classes,1,num_nodes)[:,0,:].T[unlabeled_indices]

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [77]:
np.tile(labels.T,num_samples)

ValueError: cannot reshape array of size 173950 into shape (7,1,2485)

In [21]:
yhat_[:,0,:].T.shape

(2485, 7)

In [18]:
a

array([[ 0.34883192,  0.05886821,  0.06947003, ...,  0.19176342,
         0.0854468 ,  0.09475795],
       [ 0.16398354,  0.24277214,  0.05177625, ...,  0.19748591,
         0.05157265,  0.23123692],
       [ 0.21645211,  0.06094358,  0.17534406, ...,  0.16442336,
         0.14139494,  0.11683412],
       ..., 
       [ 0.10590645,  0.03797606,  0.0667713 , ...,  0.07990669,
         0.59924595,  0.06468278],
       [ 0.09264493,  0.08716514,  0.06326002, ...,  0.09903728,
         0.1061604 ,  0.49958718],
       [ 0.42298221,  0.04664894,  0.04578981, ...,  0.20091122,
         0.0733966 ,  0.07071644]])

In [17]:
(yhat_[:,0,:].T>a-0.000001).all()

True

In [18]:
(yhat_[:,0,:].T<a+0.000001).all()

True